## Homework: Vector Search

In [7]:
from fastembed import TextEmbedding
import numpy as np

In [8]:
EMBEDDING_DIMENSIONALITY = 512
model_handle = "jinaai/jina-embeddings-v2-small-en"

# 1. Embedding the query

Embed the query: `'I just discovered the course. Can I join now?'`. Use the `'jinaai/jina-embeddings-v2-small-en'` model.

You should get a numpy array of size 512.

What's the minimal value in this array?

- 0.51
- -0.11 ✅
- 0
- 0.51

In [9]:
query = "I just discovered the course. Can I join now?"

model = TextEmbedding(model_name=model_handle, dim=EMBEDDING_DIMENSIONALITY)

q = list(model.embed(query))[0]

In [10]:
print(type(q))  
len(q) 

<class 'numpy.ndarray'>


512

In [11]:
np.min(q) 

np.float64(-0.11726373885183883)

In [12]:
np.linalg.norm(q)

np.float64(1.0)

# 2. Cosine similarity with another vector

Now let's embed this document:

```python
doc = 'Can I still join the course after the start date?'
```
What's the cosine similarity between the vector for the query and the vector for the document?

- 0.3
- 0.5
- 0.7
- 0.9 ✅

In [13]:
doc = 'Can I still join the course after the start date?'

d = list(model.embed(doc))[0]

q.dot(d)  # Dot product between query and document embeddings

np.float64(0.9008528895674548)

# 3. Ranking by cosine

Compute the embeddings for the text field, and compute the cosine between the query vector and all the documents.

What's the document index with the highest similarity? (Indexing starts from 0):

- 0
- 1 ✅
- 2
- 3
- 4

Hint: if you put all the embeddings of the text field in one matrix V (a single 2-dimensional numpy array), then computing the cosine becomes a matrix multiplication:

```V.dot(q)```

If this hint is rather confusing you than helping, feel free to ignore it.


In [18]:
documents = [
  {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
    'section': 'General course-related questions',
    'question': 'Course - Can I still join the course after the start date?',
    'course': 'data-engineering-zoomcamp'},
  {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
    'section': 'General course-related questions',
    'question': 'Course - Can I follow the course after it finishes?',
    'course': 'data-engineering-zoomcamp'},
  {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
    'section': 'General course-related questions',
    'question': 'Course - When will the course start?',
    'course': 'data-engineering-zoomcamp'},
  {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
    'section': 'General course-related questions',
    'question': 'Course - What can I do before the course starts?',
    'course': 'data-engineering-zoomcamp'},
  {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
    'section': 'General course-related questions',
    'question': 'How can we contribute to the course?',
    'course': 'data-engineering-zoomcamp'}
  ]

text_list = [doc['text'] for doc in documents]
embeddings = np.array(list(model.embed(text_list)))

In [19]:
similarities = np.dot(embeddings, q)
best_index = np.argmax(similarities)
best_index

np.int64(1)

# 4. Ranking by cosine, version two
Now let's calculate a new field, which is a concatenation of question and text:

```
full_text = doc['question'] + ' ' + doc['text']
```

Embed this field and compute the cosine between it and the query vector. What's the highest scoring document?

- 0 ✅
- 1
- 2
- 3
- 4


#### Is it different from Q3? If yes, why?

> **Yes, it's different because Q3 only uses the answer (text), while Q4 uses both the question and answer.**

In [20]:
full_text = [doc['question'] + ' ' + doc['text'] for doc in documents]
print(full_text)

full_emb = np.array(list(model.embed(full_text)))
np.argmax(full_emb.dot(q))

["Course - Can I still join the course after the start date? Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.", 'Course - Can I follow the course after it finishes? Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.', "Course - When will the course start? The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the

np.int64(0)

# 5. Selecting the embedding model

Now let's select a smaller embedding model. What's the smallest dimensionality for models in fastembed?

- 128
- 256
- 384 ✅
- 512

One of these models is `BAAI/bge-small-en`. Let's use it.

In [21]:
min_d = 512

for model in TextEmbedding.list_supported_models():
    if model['dim'] < min_d:
        print(model['model'], model['dim'])
        min_d = model['dim']
        break

BAAI/bge-small-en 384


# 6. Indexing with qdrant (2 points)
What's the highest score in the results? (The score for the first returned record):

- 0.97
- 0.87 ✅
- 0.77
- 0.67

In [22]:
from qdrant_client import QdrantClient, models

In [23]:
client = QdrantClient("http://localhost:6333")

In [24]:
import requests 

docs_url = 'https://raw.githubusercontent.com/milanimcgraw/LLM-Zoomcamp-2025/refs/heads/main/Data/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [25]:
documents[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp'}

### Choose Embedding Model

In [26]:
homework_model_handle = "BAAI/bge-small-en"

### Create Collection

In [27]:
collection_name = "vector search hw"

# Create the collection with specified vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

True

### Create, Embed & Insert Points into the Collection

In [28]:
points = []
id = 0

for course in documents_raw:
    for doc in course['documents']:
        text = doc['question'] + " " + doc['text']  

        point = models.PointStruct(
            id=id,
            vector=models.Document(text=text, model=model_handle),
            payload={
                "question": doc["question"],
                "text": doc["text"],
                "section": doc["section"],
                "course": course["course"]
            }
        )

        points.append(point)
        id += 1

In [29]:
client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

### Similiarity Search

In [30]:
def search(query, limit=1):

    results = client.query_points(
        collection_name=collection_name,
        query=models.Document( 
            text=query,
            model=model_handle 
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

In [31]:
query = "I just discovered the course. Can I join now?"

results = search(query, limit=1)

print(f"Highest score: {results.points[0].score:.4f}")
print(f"Question: {results.points[0].payload['question']}")
print(f"Text: {results.points[0].payload['text']}")

Highest score: 0.8621
Question: The course has already started. Can I still join it?
Text: Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.
In order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.
